# MIRI PSF Photometry With Space_Phot

**Author**: Ori Fox+Raphael Baer-Way
<br>
**Last Updated**: August, 2023

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
3. [Bright, Single Object](#bso)<br>
    3.1 [Multiple, Level2 Files](#bso2)<br>
    3.2 [Single, Level3 Mosaicked File](#bso3)<br>
4. [Faint/Upper Limit, Single Object](#fso)<br>
    4.1 [Multiple, Level2 Files](#fso2)<br>
    4.2 [Single, Level3 Mosaicked File](#fso3)<br>
5. [Stellar Field (LMC)](#lmv)<br>
    5.1 [Multiple, Level2 Files](#lmc2)<br>
    5.2 [Single, Level3 Mosaicked File](#lmc3)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

**Packages to Install**:
space_phot https://github.com/jpierel14/space_phot
photutils (on main git+https://github.com/astropy/photutils)
jupyter

**Goals**: 

PSF Photometry can be obtained using:

* grid of PSF models from WebbPSF
* single effective PSF (ePSF) NOT YET AVAILABLE
* grid of effective PSF NOT YET AVAILABLE

The notebook shows:

* how to obtain the PSF model from WebbPSF (or build an ePSF)
* how to perform PSF photometry on the image

**Data**: 

MIRI Data PID 1028 (Calibration Program):

jwst_download.py -v --config jwst_query.cfg --propID 1028 --obsnums 6 7 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --calib_level 3 --filetypes '_i2d.fits'

jwst_download.py -v --config jwst_query.cfg --propID 1028 --obsnums 6 7 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --calib_level 2 --filetypes '_cal.fits'


jwst_download.py -v --config jwst_query.cfg --propID 1028 --obsnums 6 7 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --filetypes '_asn.json


MIRI Data PID 1171 (LMC)


jwst_download.py -v --config jwst_query.cfg --propID 1171 --obsnums 4 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --calib_level 3 --filetypes '_i2d.fits'


jwst_download.py -v --config jwst_query.cfg --propID 1171 --obsnums 4 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --calib_level 2 --filetypes '_cal.fits'


jwst_download.py -v --config jwst_query.cfg --propID 1171 --obsnums 4 --outrootdir ./mydownloads/ -l 500 --i miri --obsmode image --token YOUR_TOKEN_HERE --filetypes '_asn.json'

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
%matplotlib inline

In [ ]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import math

import st_phot

# JWST models
#
from jwst import datamodels, associations
from jwst.datamodels import ImageModel, dqflags

# Background and PSF Functions
#
from photutils.background import MMMBackground, MADStdBackgroundRMS, Background2D
from photutils.detection import DAOStarFinder
from photutils import EPSFBuilder, GriddedPSFModel
from photutils.psf import DAOGroup, extract_stars, IterativelySubtractedPSFPhotometry

# Photutils library and tools
#
import photutils
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils import Background2D, MedianBackground, ModeEstimatorBackground, MMMBackground

import st_phot

3.<font color='white'>-</font>Bright, Single Object<a class="anchor" id="bso"></a>
------------------

### 3.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="bso2"></a> ###

In [ ]:
### Level 3 Files
lvl3 = ['mast/01028/obsnum06/jw01028-o006_t001_miri_f770w_i2d.fits']
lvl3

In [ ]:
### Create Level 2 Data List from ASN files
prefix = "mast/01028/obsnum06/"
with open(prefix+"jw01028-o006_20230527t163328_image3_00004_asn.json","r") as fi:
    lvl2 = []
    for ln in fi:
        #print(ln)
        if ln.startswith('                    "expname":'):
            x = ln[2:].split(':')
            y = x[1].split('"')
            lvl2.append(prefix+y[1])
print(lvl2)

In [ ]:
# Examine the First Image
ref_image = 'jw01028006001_02101_00001_mirimage_cal.fits'
print(ref_image)
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Zoom in to see the source
source_location = SkyCoord('17:43:04.4879','+66:55:01.837',unit=(u.hourangle,u.deg))
ref_x,ref_y = skycoord_to_pixel(source_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(21,21),(ref_y,ref_x))#Flipped x and y for dolphot-otherwise this won't work
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('PID1028,Obs006')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

### Dolphot Instruction

Now that we have all the data and have examined our source, we can proceed to running dolphot. Make sure you do not run any code altering the original fits files before running dolphot. For a general guide on installation/how to get things up and running refer here: https://github.com/18rway/dolphot 

Go to the terminal and move to the folder where the level 2 files are located. Ensure you have the PSFs for all MIRI filters you will be working with loaded in to the dolphot directory. Run the pre-processing steps: mirimask, splitgroups and calcsky. Use calcsky parameters 
### rin=10 rout=25 step=4 sigmalow=2.25 sigmahigh=2 


Now edit the parameter file(taken from your dolphot root directory) such that you are running on Nimg=1 with that one image being the level 2 file you wish to run on(for comparisons sake, use the first image in the list created in the above cells). Use the photsec parameter as outlined in the github with the ref_x and ref_y we just found, but flip the values for dolphot!(For some reason, dolphot flips the x and y astropy find). Alternatively, you can run on every level 2 file for one source(running on all filter at once) using photsec in a matter of minutes. In h\the terminal, run 
### dolphot 1028_star.phot -pdolphot.param or similar.

Now you have your dolphot output(this should take maximally 2 minutes if you specify a small region around the source with the photsec parameter). Return to this notebook, but ensure you place the output file(1028_star.phot) in your current directory from which you run this notebook.

In [ ]:
#Run this file to see your output overlayed on the image
data=np.loadtxt('1028_star.phot')#load in photometry file as whateever it is named
#print(ref_x,ref_y)
#2nd column x,3rd column y
#15th column magnitude
data_new=data[np.where((data[:,2]<ref_x+2)&(data[:,2]>ref_x-2)&(data[:,3]>ref_y-2)&(data[:,3]<ref_y+2))]# filter image by detections near source
#print(np.shape(data_new))#find number of detections
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
for i in range(len(data_new)):
    plt.text(ref_x-data_new[i,2]+10.2,ref_y-data_new[i,3]+10.2,'{}'.format(data_new[i,15]),color='r')#add magnitudes
    plt.scatter(ref_x-data_new[i,2]+10,ref_y-data_new[i,3]+10,color='r')#plot all points add 10 as a center of images is 10,10
plt.legend
print('We see the detection centered on the star. You can manually check your dolphot output for error and other measures(sharpness,roundness)')

### Repeat this process for any other level 2, level 3 images-Compare to stphot, but remember to convert from AB to Vegamag!(What Dolphot outputs)

#### F770w conversion=4.352
#### F1000w conversion=4.928
#### F1130w conversion=5.223
#### F1280w conversion=5.475
#### F1500w conversion=5.828
#### F1800w conversion=6.204
#### F2100 conversion=6.488
#### F2550 conversion=6.975


In [ ]:
# Get PSF from WebbPSF
jwst_obs = st_phot.observation2(lvl2)
psfs = st_phot.get_jwst_psf(jwst_obs,source_location,num_psfs=4)
plt.imshow(psfs[0].data)
plt.show()

In [ ]:
# Do PSF Photometry using st_phot (details of fitting are in documentation)
# https://st-phot.readthedocs.io/en/latest/examples/plot_a_psf.html#jwst-images

jwst_obs.psf_photometry(psfs,source_location,bounds={'flux':[-1000,100000],
                        'centroid':[-2,2],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=True,
                        fit_flux='single')
jwst_obs.plot_psf_fit()
plt.show()

jwst_obs.plot_psf_posterior(minweight=.0005)
plt.show()

print(jwst_obs.psf_result.phot_cal_table)

In [ ]:
# Calculate Average Magnitude from Table
mag_arr = jwst_obs.psf_result.phot_cal_table['mag']
magerr_arr = jwst_obs.psf_result.phot_cal_table['magerr']

mag_lvl2psf = np.mean(mag_arr)
magerr_lvl2psf = math.sqrt(sum(p**2 for p in magerr_arr))
print(round(mag_lvl2psf,4),round(magerr_lvl2psf,4))
#return math.sqrt(sum((p[0]-centroid[0])**2 + (p[1]-centroid[1])**2 for p in points))


### 3.2<font color='white'>-</font>Single, Level3 Mosaicked File<a class="anchor" id="bso3"></a> ###

### For level 3 images, specify img0 as the drizzled file and run with img1 still being the level 2 file. In the output the first set of photometry results(columns 15-20) will correspond to the reference drizzled image

In [ ]:
lvl3

In [ ]:
# Now do the same photometry on the Level 3 Data
ref_image = lvl3[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',
                      norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
source_location = SkyCoord('17:43:04.4879','+66:55:01.837',unit=(u.hourangle,u.deg))

ref_y,ref_x = skycoord_to_pixel(source_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('PID1028,Obs006 (level 3)')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Get PSF from WebbPSF and drizzle it to the source location

jwst3_obs = st_phot.observation3(lvl3[0])
psf3 = st_phot.get_jwst3_psf(jwst_obs,source_location,num_psfs=4)
plt.imshow(psf3.data)
plt.show()

In [ ]:
jwst3_obs.psf_photometry(psf3,source_location,bounds={'flux':[-1000,10000],
                        'centroid':[-2,2],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=True,
                        fit_flux=True)

jwst_obs.plot_psf_fit()
plt.show()

jwst_obs.plot_psf_posterior(minweight=.0005)
plt.show()

In [ ]:
mag_lvl3psf = jwst3_obs.psf_result.phot_cal_table['mag'][0]
magerr_lvl3psf = jwst3_obs.psf_result.phot_cal_table['magerr'][0]
print(round(mag_lvl2psf,4),round(magerr_lvl2psf,4))
print(round(mag_lvl3psf,5),round(magerr_lvl3psf,5))

4.<font color='white'>-</font>Faint/Upper Limit, Single Object<a class="anchor" id="fso"></a>
------------------

### Dolphot does not do upper limits in the way other methods are able to. For the purpose of comparison, you could simply extract the 5 sigma detections from some radius around the location you are performing forced photometry at and average these. However, this is not robust.

### 4.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="fso2"></a> ###

In [ ]:
### Level 3 Files
lvl3 = 'jw01028-o006_t001_miri_f770w_i2d.fits'
lvl3
lvl2='jw01028006001_02101_00001_mirimage_cal.fits'

In [ ]:
### Create Level 2 Data List from ASN files
prefix = "mast/01028/obsnum06/"
with open(prefix+"jw01028-o006_20230527t163328_image3_00004_asn.json","r") as fi:
    lvl2 = []
    for ln in fi:
        #print(ln)
        if ln.startswith('                    "expname":'):
            x = ln[2:].split(':')
            y = x[1].split('"')
            lvl2.append(prefix+y[1])
print(lvl2)

In [ ]:
# Change all DQ flagged pixels to NANs
for file in lvl2:
    hdul = fits.open(file, mode='update')
    data = fits.open(file)['SCI',1].data
    dq = fits.open(file)['DQ',1].data
    data[dq == 1]=np.nan
    hdul['SCI',1].data=data
    hdul.flush()

In [ ]:
# Examine the First Image
ref_image = lvl2[0]
print(ref_image)
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Pick a blank part of the sky to calculate the upper limit

source_location = SkyCoord('17:43:00.0332','+66:54:42.677',unit=(u.hourangle,u.deg))
ref_y,ref_x = skycoord_to_pixel(source_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('PID1028,Obs006')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Get PSF from WebbPSF
jwst_obs = space_phot.observation2(lvl2[0])
psfs = space_phot.get_jwst_psf(jwst_obs,source_location)#,num_psfs=4)
plt.imshow(psfs[0].data)
plt.show()

In [ ]:
# Do PSF Photometry using space_phot (details of fitting are in documentation)
# https://st-phot.readthedocs.io/en/latest/examples/plot_a_psf.html#jwst-images

jwst_obs.psf_photometry(psfs,source_location,bounds={'flux':[-10,1000],
                        #'centroid':[-2,2],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=True,
                        fit_centroid='fixed',
                        fit_flux='single')
jwst_obs.plot_psf_fit()
plt.show()

jwst_obs.plot_psf_posterior(minweight=.0005)
plt.show()

print(jwst_obs.psf_result.phot_cal_table)

In [ ]:
# Print Upper Limits

magupper_lvl2psf = jwst_obs.upper_limit(nsigma=5)
magupper_lvl2psf

### 4.2<font color='white'>-</font>Single, Level3 Mosaicked File<a class="anchor" id="fso3"></a> ###

In [ ]:
lvl3

In [ ]:
# Now do the same photometry on the Level 3 Data
ref_image = lvl3
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',
                      norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Pick a blank part of the sky to calculate the upper limit

source_location = SkyCoord('17:43:00.0332','+66:54:42.677',unit=(u.hourangle,u.deg))

ref_y,ref_x = skycoord_to_pixel(source_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('PID1028,Obs006 (level 3)')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
jwst3_obs = space_phot.observation3(lvl3)
psf3 = space_phot.get_jwst3_psf(jwst3_obs,source_location)#,num_psfs=4)
plt.imshow(psf3.data)
plt.show()

In [ ]:
jwst3_obs.psf_photometry(psf3,source_location,bounds={'flux':[-10,1000],
                        #'centroid':[-2,2],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=False,
                        fit_centroid=False,
                        fit_flux=True)

jwst3_obs.plot_psf_fit()
plt.show()

jwst3_obs.plot_psf_posterior(minweight=.0005)
plt.show()

In [ ]:
magupper_lvl3psf = jwst3_obs.upper_limit(nsigma=5)
print(round(magupper_lvl2psf[0],4))
print(round(magupper_lvl3psf[0],5))

5.<font color='white'>-</font>Stellar Field (LMC)<a class="anchor" id="lmc"></a>
------------------

### 5.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="lmc2"></a> ###

##### Now do the same thing for a larger group of stars and test for speed

In [ ]:
### Level 3 Files
lvl3 = ['mastDownload/JWST/jw01171-o004_t001_miri_f560w/jw01171-o004_t001_miri_f560w_i2d.fits']
lvl3

In [ ]:
### Level 2 Files
lvl2 = glob.glob('mastDownload/JWST/jw01171004*/*cal.fits')
lvl2

### Running dolphot on a field

Run dolphot in the same way while leaving the photsec parameter blank(dolphot will run across the entire file by default), but this time make the img1 one of these 1171 images. You will get a magnitude for every star in the field, which can be stored in a file you could call lmc.phot. Use the same process with a selected RA/DEC as above to isolate stars and get their magnitudes. Again, we can compare with the known AV to Vegamag conversions. I provide an example for one star

In [ ]:
# Zoom in to see the source
source_location = SkyCoord('5:23:01.9403','-69:26:46.808',unit=(u.hourangle,u.deg))#set source location for one star

ref_x,ref_y = skycoord_to_pixel(source_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(21,21),(ref_y,ref_x))#Flipped x and y for dolphot-otherwise this won't work
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('PID1028,Obs006')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
#Run this file to see your output overlayed on the image
data=np.loadtxt('lmc.phot')#load in photometry file
#print(ref_x,ref_y)
#2nd column x,3rd column y
#15th column magnitude
data_new=data[np.where((data[:,2]<ref_x+2)&(data[:,2]>ref_x-2)&(data[:,3]>ref_y-2)&(data[:,3]<ref_y+2))]# filter image by detections near source
#print(np.shape(data_new))#find number of detections
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
for i in range(len(data_new)):
    plt.text(ref_x-data_new[i,2]+10.2,ref_y-data_new[i,3]+10.2,'{}'.format(data_new[i,15]),color='r')#add magnitudes
    plt.scatter(ref_x-data_new[i,2]+10,ref_y-data_new[i,3]+10,color='r')#plot all points add 10 as a center of images is 10,10
plt.legend
print('We see the detection centered on the star. You can manually check your dolphot output for error and other measures(sharpness,roundness)')

In [ ]:
# Find Stars in Level 3 File

# Get rough estimate of background (There are Better Ways to Do Background Subtraction)
bkgrms = MADStdBackgroundRMS()
mmm_bkg = MMMBackground()

im = fits.open(lvl3[0]) 
w = wcs.WCS(im['SCI',1])

std = bkgrms(im[1].data)
bkg = mmm_bkg(im[1].data)
data_bkgsub = im[1].data.copy()
data_bkgsub -= bkg        
sigma_psf = 1.636 #pixls for F770W
threshold = 5.

daofind = DAOStarFinder(threshold=threshold * std, fwhm=sigma_psf, exclude_border=True)
found_stars = daofind(data_bkgsub)

In [ ]:
found_stars.pprint_all(max_lines=10)

In [ ]:
# Filter out only stars you want

plt.figure(figsize=(12, 8))
plt.clf()

ax1 = plt.subplot(2, 1, 1)

ax1.set_xlabel('mag')
ax1.set_ylabel('sharpness')

xlim0 = np.min(found_stars['mag']) - 0.25
xlim1 = np.max(found_stars['mag']) + 0.25
ylim0 = np.min(found_stars['sharpness']) - 0.15
ylim1 = np.max(found_stars['sharpness']) + 0.15

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

#ax1.xaxis.set_major_locator(ticker.AutoLocator())
#ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
#ax1.yaxis.set_major_locator(ticker.AutoLocator())
#ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.scatter(found_stars['mag'], found_stars['sharpness'], s=10, color='k')

sh_inf = 0.40
sh_sup = 0.82
#mag_lim = -5.0
lmag_lim = -3.0
umag_lim = -5.0

ax1.plot([xlim0, xlim1], [sh_sup, sh_sup], color='r', lw=3, ls='--')
ax1.plot([xlim0, xlim1], [sh_inf, sh_inf], color='r', lw=3, ls='--')
ax1.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')
ax1.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')

ax2 = plt.subplot(2, 1, 2)

ax2.set_xlabel('mag')
ax2.set_ylabel('roundness')

ylim0 = np.min(found_stars['roundness2']) - 0.25
ylim1 = np.max(found_stars['roundness2']) - 0.25

ax2.set_xlim(xlim0, xlim1)
ax2.set_ylim(ylim0, ylim1)

#ax2.xaxis.set_major_locator(ticker.AutoLocator())
#ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
#ax2.yaxis.set_major_locator(ticker.AutoLocator())
#ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())

round_inf = -0.40
round_sup = 0.40

ax2.scatter(found_stars['mag'], found_stars['roundness2'], s=10, color='k')

ax2.plot([xlim0, xlim1], [round_sup, round_sup], color='r', lw=3, ls='--')
ax2.plot([xlim0, xlim1], [round_inf, round_inf], color='r', lw=3, ls='--')
ax2.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')
ax2.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')

plt.tight_layout()

In [ ]:
mask = ((found_stars['mag'] < lmag_lim) & (found_stars['mag'] > umag_lim) & (found_stars['roundness2'] > round_inf)
        & (found_stars['roundness2'] < round_sup) & (found_stars['sharpness'] > sh_inf) 
        & (found_stars['sharpness'] < sh_sup) & (found_stars['xcentroid'] > 100) & (found_stars['xcentroid'] < 950)
        & (found_stars['ycentroid'] > 100) & (found_stars['ycentroid'] < 950))

found_stars_sel = found_stars[mask]

print('Number of stars found originally:', len(found_stars))
print('Number of stars in final selection:', len(found_stars_sel))


In [ ]:
found_stars_sel

In [ ]:
# Convert pixel to wcs coords
from astropy.wcs.utils import skycoord_to_pixel
skycoords = w.pixel_to_world(found_stars_sel['xcentroid'], found_stars_sel['ycentroid'])
skycoords = skycoords[22:]
len(skycoords)

In [ ]:
lvl3

In [ ]:
lvl2

In [ ]:
file = lvl2[0]
dq = fits.open(file)['DQ',1].data
dq[233, 340]

In [ ]:
# Developer note. Would be great to have a fast/approximate look up table.
jwst_obs = st_phot.observation2(lvl2)
grid = st_phot.util.get_jwst_psf_grid(jwst_obs,num_psfs=4)

In [ ]:
# Now Loop Through All Stars and Build Photometry Table
import pandas as pd
counter = 0.
badindex = []

for source_location in skycoords:
    #print(source_location)
    tic = time.perf_counter()
    print('Starting',counter+1., ' of', len(skycoords), ':',  source_location)
    #psfs = st_phot.get_jwst_psf(jwst_obs,source_location,num_psfs=4)
    psfs = st_phot.util.get_jwst_psf_from_grid(jwst_obs,source_location,grid)
    jwst_obs.psf_photometry(psfs,source_location,bounds={'flux':[-1000,100000],
                        'centroid':[-2.,2.],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=True,
                        fit_flux='single',
                        maxiter=5000)
    
    jwst_obs.plot_psf_fit()
    plt.show()
#
#    jwst_obs.plot_psf_posterior(minweight=.0005)
#    plt.show()
    
    ra = jwst_obs.psf_result.phot_cal_table['ra'][0]
    dec = jwst_obs.psf_result.phot_cal_table['dec'][0]
    mag_arr = jwst_obs.psf_result.phot_cal_table['mag']
    magerr_arr = jwst_obs.psf_result.phot_cal_table['magerr']
    mag_lvl2psf = np.mean(mag_arr)
    magerr_lvl2psf = math.sqrt(sum(p**2 for p in magerr_arr))

    #print('Finished',counter, ' of', len(skycoords), ':',  source_location)
    if counter == 0:
        df = pd.DataFrame(np.array([[ra, dec, mag_lvl2psf, magerr_lvl2psf]]), columns=['ra','dec','mag','magerr'])
    else:
        df = pd.concat([df, pd.DataFrame(np.array([[ra, dec, mag_lvl2psf, magerr_lvl2psf]]))], ignore_index=True)
    counter = counter + 1.
    toc = time.perf_counter()
    print("Elapsed Time for Photometry:", toc - tic)

### 5.2<font color='white'>-</font>Single, Level3 Mosaicked File<a class="anchor" id="lmc3"></a> ###

In [ ]:
lvl3

In [ ]:
# Now do the same photometry on the Level 3 Data
ref_image = lvl3[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',
                      norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
# Get PSF from WebbPSF and drizzle it to the source location
# Develop Note: Need Grid Capability for Level3 Data
jwst3_obs = st_phot.observation3(lvl3[0])
#psf3 = st_phot.get_jwst3_psf(jwst_obs,source_location,num_psfs=4)
#grid = st_phot.util.get_jwst_psf_grid(jwst_obs,num_psfs=4)

In [ ]:
# Now Loop Through All Stars and Build Photometry Table
import pandas as pd
counter = 0.
badindex = []

for source_location in skycoords:
    #print(source_location)
    tic = time.perf_counter()
    print('Starting',counter+1., ' of', len(skycoords), ':',  source_location)
    #psf3 = st_phot.util.get_jwst_psf_from_grid(jwst3_obs,source_location,grid)
    psf3 = st_phot.get_jwst3_psf(jwst_obs,source_location,num_psfs=4)
    jwst3_obs.psf_photometry(psf3,source_location,bounds={'flux':[-1000,10000],
                        'centroid':[-2,2],
                        'bkg':[0,50]},
                        fit_width=5,
                        fit_bkg=True,
                        fit_flux=True)

    jwst3_obs.plot_psf_fit()
    plt.show()

    ra = jwst3_obs.psf_result.phot_cal_table['ra'][0]
    dec = jwst3_obs.psf_result.phot_cal_table['dec'][0]
    mag_lvl3psf = jwst3_obs.psf_result.phot_cal_table['mag'][0]
    magerr_lvl3psf = jwst3_obs.psf_result.phot_cal_table['magerr'][0]

    #print('Finished',counter, ' of', len(skycoords), ':',  source_location)
    if counter == 0:
        df = pd.DataFrame(np.array([[ra, dec, mag_lvl3psf, magerr_lvl3psf]]), columns=['ra','dec','mag','magerr'])
    else:
        df = pd.concat([df, pd.DataFrame(np.array([[ra, dec, mag_lvl3psf, magerr_lvl3psf]]))], ignore_index=True)
    counter = counter + 1.
    toc = time.perf_counter()
    print("Elapsed Time for Photometry:", toc - tic)

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>